In [34]:

import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed


In [41]:

y = pd.read_csv('../../db/data/secondary/2010/logs_.csv', index_col=0)

y.reset_index(inplace=True, drop=True)

y

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,0.144604,0.034409,0.074476,0.207909,-0.092540,-0.115280,-0.073429,0.036865,-0.102837,0.048681,...,0.052762,0.186800,-0.112200,-0.060710,-0.181493,-0.140142,-0.296141,0.233651,0.121514,0.161598
1,-0.158431,0.019349,-0.060055,0.063573,-0.151811,0.180953,0.099397,-0.069270,0.019447,-0.138100,...,-0.104216,0.021033,-0.186871,-0.222028,-0.042750,-0.119421,-0.213643,-0.009526,0.168520,0.086077
2,-0.280275,-0.075269,0.007208,0.012066,0.122801,0.203804,0.299857,0.092724,0.040763,-0.235952,...,0.020051,0.026845,0.131037,-0.308742,-0.104275,-0.005975,0.102714,-0.408581,-0.202263,-0.116219
3,0.170612,0.036565,-0.103367,0.051466,0.023428,-0.124045,-0.266003,-0.155339,-0.087192,0.077600,...,-0.268345,-0.113249,-0.138498,0.115388,-0.004013,-0.146503,-0.027749,0.144342,0.018316,-0.123903
4,0.009589,0.043002,0.036071,-0.032162,0.046837,0.032287,0.029341,-0.005592,-0.020083,-0.004013,...,0.000000,-0.061828,0.013672,0.041085,0.068196,0.055336,0.070741,-0.037262,-0.006748,-0.016172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,-0.181807,-0.151281,-0.067330,0.000000,-0.079661,0.027265,0.109512,0.091666,-0.000764,-0.181132,...,0.028220,0.094455,0.015581,-0.179951,-0.151245,-0.061184,-0.080009,-0.101213,-0.074920,0.007244
6140,-0.015824,-0.022814,0.028862,0.038825,-0.010958,-0.004194,0.040123,0.037108,0.129765,0.105695,...,0.078348,0.084637,0.041537,-0.055637,-0.052779,-0.008054,-0.041362,-0.010921,-0.010410,0.033799
6141,0.050433,-0.010371,0.038470,-0.055259,0.077676,-0.056632,-0.008509,0.046364,-0.027466,0.020372,...,0.023492,-0.066150,0.071354,0.108596,0.048611,0.098206,0.136029,-0.032770,-0.093731,-0.042251
6142,-0.036588,-0.043548,-0.044490,0.061222,-0.024890,-0.004194,-0.025533,-0.056808,0.009156,0.002264,...,-0.042296,0.079273,-0.024642,-0.088745,-0.119423,-0.120782,-0.088009,-0.036746,-0.015621,-0.019324


In [53]:
X = pd.read_csv('../../db/data/tertiary/2010/logs_.csv', index_col=0)

X.shape

(6144, 8)

In [44]:

train = 0.7
valid = 0.9


x_n = len(X)
y_n = len(y)

X_, y_ = {}, {}

X_['train'] = X[0:int(x_n*train)]
X_['valid'] = X[int(x_n*train):int(x_n*valid)]
X_['test']  = X[int(x_n*valid):]

y_['train'] = y[0:int(y_n*train)]
y_['valid'] = y[int(y_n*train):int(y_n*valid)]
y_['test']  = y[int(y_n*valid):]

num_features = X.shape[1]

num_features

X_

{'train':                                 AUD       CAD       CHF       EUR       GBP  \
 2010-01-04 00:00:00+00:00  0.061881 -0.084664  0.028045 -0.037279  0.040824   
 2010-01-04 01:00:00+00:00 -0.033417  0.125230 -0.050101 -0.012460 -0.073844   
 2010-01-04 02:00:00+00:00 -0.032196  0.252279  0.047114  0.011464 -0.026092   
 2010-01-04 03:00:00+00:00  0.052931 -0.112991  0.009011 -0.033590 -0.107902   
 2010-01-04 04:00:00+00:00  0.020095  0.010240 -0.020462  0.002050 -0.016285   
 ...                             ...       ...       ...       ...       ...   
 2010-09-09 23:00:00+00:00 -0.019957  0.013731  0.014446 -0.058569 -0.035316   
 2010-09-10 00:00:00+00:00  0.073803  0.149694 -0.059562  0.012906 -0.010852   
 2010-09-10 01:00:00+00:00  0.114687  0.041749 -0.362267 -0.015757  0.075065   
 2010-09-10 02:00:00+00:00 -0.153586 -0.053709  0.261702 -0.022577 -0.023860   
 2010-09-10 03:00:00+00:00 -0.071138 -0.031101  0.037765  0.046331  0.003691   
 
                             

In [54]:
n_lag = 1
n_neurons = 200
n_epochs = 1000
n_batch = 1
n_neurons = 1

# design network
model = Sequential()

model.add(LSTM(n_neurons, stateful=True))

model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mse')

In [56]:
model.fit(x=X_['train'], y=y_['train'])

ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra